In [1]:
%%writefile pergola.yaml
version: v1

components:
- name: todo-app
  docker:
    file: Dockerfile
  ports:
  - 5050
  ingresses:
  - host: app
  env:
  - name: SQLALCHEMY_HOST
    component-ref: todo-db
  - name: SQLALCHEMY_PORT
    value: "5432"
  - name: SQLALCHEMY_DB
    value: flask-demo
  - name: SQLALCHEMY_USER
    config-ref: DB_USER
  - name: SQLALCHEMY_PASSWORD
    config-ref: DB_PASS

- name: todo-db
  docker:
    image: postgres:14.2
  env:
  - name: POSTGRES_DB
    value: flask-demo
  - name: POSTGRES_USER
    config-ref: DB_USER
  - name: POSTGRES_PASSWORD
    config-ref: DB_PASS
  storage:
  - name: pgdata
    path: /var/lib/postgresql/data
    size: 1Gi
  ports:
  - 5432

Writing pergola.yaml


In [2]:
%%writefile Dockerfile
FROM python:3.8-slim

RUN apt-get update -y && \
    apt-get install -y gcc libpq-dev && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

WORKDIR /todo-list

COPY test.py test.py

ENV FLASK_APP=app
CMD flask run --host=0.0.0.0 --port 5050

Writing Dockerfile


In [19]:
%%writefile docker-compose.yml
version: '3'
services:
  flask-todo-web:
    build: .
    environment:
      SQLALCHEMY_HOST: flask-todo-db
      SQLALCHEMY_PORT: "5432"
      SQLALCHEMY_DB: flask-demo
      SQLALCHEMY_USER: flask-demo
      SQLALCHEMY_PASSWORD: flask-demo
    ports:
    - "5050:5050"
    depends_on:
    - flask-todo-db

  flask-todo-db:
    image: postgres:14.2
    environment:
      POSTGRES_USER: flask-demo
      POSTGRES_PASSWORD: flask-demo
      POSTGRES_DB: flask-demo
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -h localhost -U flask-demo"]
      interval: 2s
      timeout: 5s
      retries: 5
    ports:
    - "5432:5432"

Overwriting docker-compose.yml


In [12]:
!ls

Dockerfile                     pergola.yaml
README                         pergola_getting_familiar.ipynb
docker-compose.yml             requirements.txt
olip78.git                     test.py


In [23]:
%%sh
git init
git remote add origin https://github.com/olip78/pergola_test.git
git pull origin main
git add --all
git commit -m "third commit"
git branch -M main
git push origin master

Reinitialized existing Git repository in /Users/andreichekunov/brandcompete/sagemaker/pergola_deployment/test/.git/


error: remote origin already exists.
From https://github.com/olip78/pergola_test
 * branch            main       -> FETCH_HEAD
hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: refusing to merge unrelated histories


On branch main
nothing to commit, working tree clean


error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/olip78/pergola_test.git'


CalledProcessError: Command 'b'git init\ngit remote add origin https://github.com/olip78/pergola_test.git\ngit pull origin main\ngit add --all\ngit commit -m "third commit"\ngit branch -M main\ngit push origin master\n'' returned non-zero exit status 1.